In [1]:
import pybullet as p
import time
import pybullet_data

sim_freq = 300
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # optionally
p.setGravity(0, 0, -9.81)
p.setTimeStep(1.0/sim_freq)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.5]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
robot = p.loadURDF("urdf/quadrobot.urdf", startPos, startOrientation, useFixedBase=1,
                        flags=p.URDF_USE_INERTIA_FROM_FILE+p.URDF_USE_SELF_COLLISION+p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS)

pybullet build time: Dec  1 2021 18:33:43


In [ ]:
jointIds = []
for j in range(p.getNumJoints(robot)):
    p.changeDynamics(robot, j, linearDamping=0, angularDamping=0)
    info = p.getJointInfo(robot, j)
    # print(info)
    #jointName = info[1]
    jointType = info[2]
    if (jointType == p.JOINT_REVOLUTE):
        jointIds.append(j)

joint_dir = [-1]*16
# joint_offset = [0, -1.57, 0, 0, 1.57, 0, 0, 1.57, 0, 0, -1.57, 0]
joint_offset = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
from math import *

bx = 0.257
by = 0.093
bz = 0.0
L1 = 0.0905
L2 = 0.202
L3 = 0.17629

def fkine_R1(theta):
    ef_pos = [0]*3
    ef_pos[0] = bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1])
    ef_pos[1] = -by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1])
    ef_pos[2] = L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1])
    return ef_pos

def fkine_R2(theta):
    ef_pos = [0]*3
    ef_pos[0] = -(bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1]))
    ef_pos[1] = -by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1])
    ef_pos[2] = -(L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1]))
    return ef_pos

def fkine_L1(theta):
    ef_pos = [0]*3
    ef_pos[0] = bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1])
    ef_pos[1] = -(-by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1]))
    ef_pos[2] = -(L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1]))
    return ef_pos

def fkine_L2(theta):
    ef_pos = [0]*3
    ef_pos[0] = -(bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1]))
    ef_pos[1] = -(-by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1]))
    ef_pos[2] = L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1])
    return ef_pos

def fkine(theta):
    ef_pos = [0]*12
    ef_pos[0:3] = fkine_R1(theta[0:3])
    ef_pos[3:6] = fkine_L1(theta[3:6])
    ef_pos[6:9] = fkine_R2(theta[6:9])
    ef_pos[9:] = fkine_L2(theta[9:])
    return ef_pos

In [ ]:
ef_pos_theor = []
ef_pos_pract = []
cur_joint_pos_tmp = []
cur_joint_pos = []
t = 0
t_axis = []

theta_cur = []
theta_ref = []

In [ ]:
import random

A = [radians(random.randint(10, 45)) for _ in range(12)]
phi = [random.random()*3.14 for _ in range(12)]
nu = [random.randint(1, 20) for _ in range(12)]

In [ ]:
ref_pos=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

for i in range (1000):
    for i in range(12):
        ref_pos[i] = A[i]*sin(nu[i]*t + phi[i]) 

    for i in range(len(jointIds)):
            p.setJointMotorControl2(robot, jointIds[i], p.POSITION_CONTROL,
                                    targetPosition=joint_dir[i]*ref_pos[i]-joint_offset[i],
                                    maxVelocity=15.0)#
    p.stepSimulation()
    t += 1.0/sim_freq
    
    ef_pos_pract_tmp1 = p.getLinkState(robot, 4)
    ef_pos_pract_tmp2 = p.getLinkState(robot, 8)
    ef_pos_pract_tmp3 = p.getLinkState(robot, 12)
    ef_pos_pract_tmp4 = p.getLinkState(robot, 16)
    ef_pos_pract.append([[x for x in ef_pos_pract_tmp1[0]], [x for x in ef_pos_pract_tmp2[0]], 
                         [x for x in ef_pos_pract_tmp3[0]], [x for x in ef_pos_pract_tmp4[0]]])
    # ef_pos_pract_tmp = p.getLinkState(robot, 8)
    # ef_pos_pract.append([x for x in ef_pos_pract_tmp[0]])
    # ef_pos_pract_tmp = p.getLinkState(robot, 12)
    # ef_pos_pract.append([x for x in ef_pos_pract_tmp[0]])
    # ef_pos_pract_tmp = p.getLinkState(robot, 16)
    # ef_pos_pract.append([x for x in ef_pos_pract_tmp[0]])
    
    joint_states = p.getJointStates(robot, range(16))
    cur_joint_pos_tmp = ([-x[0] for x in joint_states])
    cur_joint_pos = []
    for i in range(16):
        if i%4 != 0:
            cur_joint_pos.append(cur_joint_pos_tmp[i])
    ef_pos_theor.append(fkine(cur_joint_pos))

    theta_cur.append(cur_joint_pos)
    theta_ref.append(ref_pos[0:12])
    t_axis.append(t)
    
    time.sleep(1.0/sim_freq)



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 15]


leg_num = 3
for leg_num in range(4):
    plt.figure(figsize=(10,15),
           facecolor='white')

    # plot for theta1 
    plt.subplot(3, 1, 1)
    theta1_cur = [x[0+leg_num] for x in theta_cur]
    theta1_ref = [x[0+leg_num] for x in theta_ref]
    plt.plot(t_axis, theta1_cur, color = 'r', label='Current angle')
    plt.plot(t_axis, theta1_ref, color = 'g', label='Desired angle')
    plt.legend()
    plt.grid()
    plt.ylabel("theta1 (rad)")

    # plot for theta2
    plt.subplot(3, 1, 2)
    theta2_cur = [x[1+leg_num] for x in theta_cur]
    theta2_ref = [x[1+leg_num] for x in theta_ref]
    plt.plot(t_axis, theta2_cur, color = 'r')
    plt.plot(t_axis, theta2_ref, color = 'g')
    plt.grid()
    plt.ylabel("theta2 (rad)")

    # plot for theta3
    plt.subplot(3, 1, 3)
    theta3_cur = [x[2+leg_num] for x in theta_cur]
    theta3_ref = [x[2+leg_num] for x in theta_ref]
    plt.plot(t_axis, theta3_cur, color = 'r')
    plt.plot(t_axis, theta3_ref, color = 'g')
    plt.grid()
    plt.ylabel("theta3 (rad)")

    plt.suptitle("Joint angles for leg_num={0}".format(leg_num+1))
    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 15]

leg_num = 3
for leg_num in range(4):
    plt.figure(figsize=(10,15),
            facecolor='white')
    # plot for x
    ef_pract_x = [x[0+leg_num][0] for x in ef_pos_pract]
    ef_theor_x = [x[0+leg_num*3] for x in ef_pos_theor]

    plt.subplot(3, 1, 1)
    plt.plot(t_axis, ef_pract_x, color = 'r', label='Simulated value') # pract
    plt.plot(t_axis, ef_theor_x, color = 'g', label='Calculated value') # theor
    plt.legend()
    plt.grid()
    plt.ylabel("x (m)")


    # plot for y
    ef_pract_y = [x[0+leg_num][1] for x in ef_pos_pract]
    ef_theor_y = [x[1+leg_num*3] for x in ef_pos_theor]

    plt.subplot(3, 1, 2)
    plt.plot(t_axis, ef_pract_y, color = 'r') # pract
    plt.plot(t_axis, ef_theor_y, color = 'g') # theor
    plt.grid()
    plt.ylabel("y (m)")


    # plot for z
    ef_pract_z = [x[0+leg_num][2]-0.5 for x in ef_pos_pract]
    ef_theor_z = [x[2+leg_num*3] for x in ef_pos_theor]

    plt.subplot(3, 1, 3)
    plt.plot(t_axis, ef_pract_z, color = 'r') # pract
    plt.plot(t_axis, ef_theor_z, color = 'g') # theor
    plt.ylabel("z (m)")
    plt.xlabel("time (sec)")
    plt.grid()

    plt.suptitle("Forward kinematics check for leg #{0}".format(leg_num+1))
    plt.show()